## Les formats de fichiers - TP



### Préparation d'une base de données SQLite à partir d'un fichier parquet

In [2]:
from pathlib import Path
import sqlite3
import os
import urllib.request

import pandas as pd

In [3]:
DATA_DIR = Path("../data")

Télécharger le fichier des trajets en taxi de New York en janvier 2025.

In [5]:
url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-01.parquet"
parquet_file_path = Path(DATA_DIR, "yellow_tripdata_2025-01.parquet")

if not os.path.exists(parquet_file_path):
    print("Téléchargement du fichier...")
    urllib.request.urlretrieve(url, parquet_file_path)
    print("Fichier téléchargé.")
else:
    print("Fichier déjà présent.")

Fichier déjà présent.


Lire le fichier avec pandas. Utiliser le moteur pyarrow.

In [8]:
taxi_df = pd.read_parquet(parquet_file_path, engine="pyarrow")
taxi_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
0,1,2025-01-01 00:18:38,2025-01-01 00:26:59,1.0,1.60,1.0,N,229,237,1,10.0,3.5,0.5,3.00,0.0,1.0,18.00,2.5,0.0,0.0
1,1,2025-01-01 00:32:40,2025-01-01 00:35:13,1.0,0.50,1.0,N,236,237,1,5.1,3.5,0.5,2.02,0.0,1.0,12.12,2.5,0.0,0.0
2,1,2025-01-01 00:44:04,2025-01-01 00:46:01,1.0,0.60,1.0,N,141,141,1,5.1,3.5,0.5,2.00,0.0,1.0,12.10,2.5,0.0,0.0
3,2,2025-01-01 00:14:27,2025-01-01 00:20:01,3.0,0.52,1.0,N,244,244,2,7.2,1.0,0.5,0.00,0.0,1.0,9.70,0.0,0.0,0.0
4,2,2025-01-01 00:21:34,2025-01-01 00:25:06,3.0,0.66,1.0,N,244,116,2,5.8,1.0,0.5,0.00,0.0,1.0,8.30,0.0,0.0,0.0


Créer une base sqlite3 et importer les données.

In [10]:
conn = sqlite3.connect("nyc_taxi.db")
taxi_df.to_sql("trips", conn, index=False, if_exists="append")
conn.close()

Déplacer le fichier .db dans le dossier data. Se reconnecter à la base de données et lire les 5 premières lignes.

In [11]:
conn = sqlite3.connect(Path(DATA_DIR, "nyc_taxi.db"))
df_sql = pd.read_sql("SELECT * FROM trips LIMIT 5", conn)
df_sql.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
0,1,2025-01-01 00:18:38,2025-01-01 00:26:59,1.0,1.60,1.0,N,229,237,1,10.0,3.5,0.5,3.00,0.0,1.0,18.00,2.5,0.0,0.0
1,1,2025-01-01 00:32:40,2025-01-01 00:35:13,1.0,0.50,1.0,N,236,237,1,5.1,3.5,0.5,2.02,0.0,1.0,12.12,2.5,0.0,0.0
2,1,2025-01-01 00:44:04,2025-01-01 00:46:01,1.0,0.60,1.0,N,141,141,1,5.1,3.5,0.5,2.00,0.0,1.0,12.10,2.5,0.0,0.0
3,2,2025-01-01 00:14:27,2025-01-01 00:20:01,3.0,0.52,1.0,N,244,244,2,7.2,1.0,0.5,0.00,0.0,1.0,9.70,0.0,0.0,0.0
4,2,2025-01-01 00:21:34,2025-01-01 00:25:06,3.0,0.66,1.0,N,244,116,2,5.8,1.0,0.5,0.00,0.0,1.0,8.30,0.0,0.0,0.0
